Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

# data already shuffled
with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [13]:
print(train_labels[1:5])
t_labels = train_labels[1:5]
t_labels = (np.arange(10) == t_labels[:,None]).astype(np.float32)
print(t_labels)

[9 6 2 7]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]]


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [9]:
import time
print('creating neural network...')
img_size = 28
num_labels = 10 
batch_size = 128
hidden_nodes = 1024
num_total_examples = train_dataset.shape[0] 
beta = 0.001 # hyperparameter for regularizing with L2 norm
num_steps = 1001

def neuralnet1(x, weights, biases):
    # logits 
    layer_1 = tf.matmul(x['tf_x'], weights['weights_1']) + biases['biases_1']
    layer_1 = tf.nn.relu(layer_1) 
    layer_2 = tf.matmul(layer_1, weights['weights_2']) + biases['biases_2']
    return layer_2
    
# 2 layer neural network 
graph = tf.Graph()
with graph.as_default():
    # inputs
    inputs = {
        'tf_x' : tf.placeholder(tf.float32, shape=(batch_size, img_size * img_size)),
        'tf_x_labels' : tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    }
    
    # variables
    # weights randomly initialized with normal distribution
    weights = {
        'weights_1' : tf.Variable(tf.truncated_normal([img_size * img_size, hidden_nodes])),
        'weights_2' : tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    }
    # initialize biases to zero
    biases = {
        'biases_1' : tf.Variable(tf.zeros([hidden_nodes])),
        'biases_2' : tf.Variable(tf.zeros([num_labels]))
    }
    
    # computations
    output = neuralnet1(inputs, weights, biases)
    
    #loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        labels=inputs['tf_x_labels'], logits=output)) + beta * (
        tf.nn.l2_loss(weights['weights_1']) + tf.nn.l2_loss(weights['weights_2']))
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # predictions
    tf_train_prediction = tf.nn.softmax(output)
    tf_valid_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(valid_dataset, weights['weights_1']) + biases['biases_1']), weights['weights_2']) + biases['biases_2'])
    tf_test_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(test_dataset, weights['weights_1']) + biases['biases_1']), weights['weights_2']) + biases['biases_2'])
print('graph initialized!')

#run the network
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('training started!\n')
    start = time.clock()
    for step in range(num_steps):
        offset = (batch_size * step) % (num_total_examples - batch_size)
        tf_batch_dataset = train_dataset[offset:offset + batch_size, :]
        tf_batch_labels = train_labels[offset:offset + batch_size, :]
        feed_dict = {
            inputs['tf_x'] : tf_batch_dataset, inputs['tf_x_labels'] : tf_batch_labels
        }
        _, l, pred = session.run([optimizer, loss, tf_train_prediction], feed_dict=feed_dict)
        
        # print accuracies on training data, validation data, and test data
        if step % 500 == 0:
            print('Step: %d' % step)
            print('Time elapsed:', time.clock() - start, 's')
            print('Loss: %.2f' % l)
            print('Batch training accuracy: %.1f%%' % accuracy(pred, tf_batch_labels))
            print('Batch validation accuracy: %.1f%%' % accuracy(tf_valid_prediction.eval(), valid_labels))
            print('------')
    print('Total test accuracy: %.1f%%' % accuracy(tf_test_prediction.eval(), test_labels))
    print('Total training time:', time.clock() - start, 's')
    


creating neural network...
graph initialized!
training started!
Step: 0
Time elapsed: 0.107416 s
Loss: 656.38
Batch training accuracy: 12.5%
Batch validation accuracy: 31.4%
------
Step: 500
Time elapsed: 62.67856 s
Loss: 195.02
Batch training accuracy: 82.8%
Batch validation accuracy: 81.2%
------
Step: 1000
Time elapsed: 126.658572 s
Loss: 113.80
Batch training accuracy: 89.1%
Batch validation accuracy: 82.5%
------
Total test accuracy: 89.3%
Total training time: 130.459824 s


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [16]:
num_steps = 8001

# overfitting example - results in high training accuracy, low validation and test accuracy
# run the network
overfit_dataset = train_dataset[:128*5, :]
overfit_labels = train_labels[:1000, :]
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('training started!\n')
    start = time.clock()
    for step in range(num_steps):
        offset = (batch_size * step) % (128*5 - batch_size)
        tf_batch_dataset = overfit_dataset[offset:offset + batch_size, :]
        tf_batch_labels = overfit_labels[offset:offset + batch_size, :]
        feed_dict = {
            inputs['tf_x'] : tf_batch_dataset, inputs['tf_x_labels'] : tf_batch_labels
        }
        _, l, pred = session.run([optimizer, loss, tf_train_prediction], feed_dict=feed_dict)
        
        # print accuracies on training data, validation data, and test data
        if step % 500 == 0:
            print('Step: %d' % step)
            print('Time elapsed:', time.clock() - start, 's')
            print('Loss: %.2f' % l)
            print('Batch training accuracy: %.1f%%' % accuracy(pred, tf_batch_labels))
            print('Batch validation accuracy: %.1f%%' % accuracy(tf_valid_prediction.eval(), valid_labels))
            print('------')
    print('Total test accuracy: %.1f%%' % accuracy(tf_test_prediction.eval(), test_labels))
    print('Total training time:', time.clock() - start, 's')

training started!

Step: 0
Time elapsed: 0.12415 s
Loss: 1062.12
Batch training accuracy: 9.4%
Batch validation accuracy: 17.0%
------
Step: 500
Time elapsed: 53.95906 s
Loss: 200.40
Batch training accuracy: 100.0%
Batch validation accuracy: 77.2%
------
Step: 1000
Time elapsed: 107.25771 s
Loss: 121.54
Batch training accuracy: 100.0%
Batch validation accuracy: 78.3%
------
Step: 1500
Time elapsed: 161.057263 s
Loss: 74.23
Batch training accuracy: 100.0%
Batch validation accuracy: 77.8%
------
Step: 2000
Time elapsed: 217.904439 s
Loss: 45.35
Batch training accuracy: 100.0%
Batch validation accuracy: 78.4%
------
Step: 2500
Time elapsed: 277.493794 s
Loss: 27.98
Batch training accuracy: 100.0%
Batch validation accuracy: 77.8%
------
Step: 3000
Time elapsed: 342.008394 s
Loss: 17.22
Batch training accuracy: 100.0%
Batch validation accuracy: 77.8%
------
Step: 3500
Time elapsed: 406.878018 s
Loss: 10.86
Batch training accuracy: 100.0%
Batch validation accuracy: 78.0%
------
Step: 4000
Ti

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [17]:
# introducing dropout 

def dropout_neuralnet1(x, weights, biases):
    # logits 
    layer_1 = tf.matmul(x['tf_x'], weights['weights_1']) + biases['biases_1']
    layer_1 = tf.nn.relu(layer_1) 
    layer_1 = tf.nn.dropout(layer_1, 0.5)
    layer_2 = tf.matmul(layer_1, weights['weights_2']) + biases['biases_2']
    layer_2 = tf.nn.dropout(layer_2, 0.5)
    return layer_2

graph = tf.Graph()
with graph.as_default():
    # inputs
    inputs = {
        'tf_x' : tf.placeholder(tf.float32, shape=(batch_size, img_size * img_size)),
        'tf_x_labels' : tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    }
    
    # variables
    # weights randomly initialized with normal distribution
    weights = {
        'weights_1' : tf.Variable(tf.truncated_normal([img_size * img_size, hidden_nodes])),
        'weights_2' : tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    }
    # initialize biases to zero
    biases = {
        'biases_1' : tf.Variable(tf.zeros([hidden_nodes])),
        'biases_2' : tf.Variable(tf.zeros([num_labels]))
    }
    
    # computations
    output = neuralnet1(inputs, weights, biases)
    dropout = dropout_neuralnet1(inputs, weights, biases)
    
    #loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        labels=inputs['tf_x_labels'], logits=dropout)) + beta * (
        tf.nn.l2_loss(weights['weights_1']) + tf.nn.l2_loss(weights['weights_2']))
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # predictions
    tf_train_prediction = tf.nn.softmax(output)
    tf_valid_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(valid_dataset, weights['weights_1']) + biases['biases_1']), weights['weights_2']) + biases['biases_2'])
    tf_test_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(test_dataset, weights['weights_1']) + biases['biases_1']), weights['weights_2']) + biases['biases_2'])
print('graph initialized!')

# overfitting example - results in high training accuracy, low validation and test accuracy
# run the network
overfit_dataset = train_dataset[:128*5, :]
overfit_labels = train_labels[:1000, :]
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('training started!\n')
    start = time.clock()
    for step in range(num_steps):
        offset = (batch_size * step) % (128*5 - batch_size)
        tf_batch_dataset = overfit_dataset[offset:offset + batch_size, :]
        tf_batch_labels = overfit_labels[offset:offset + batch_size, :]
        feed_dict = {
            inputs['tf_x'] : tf_batch_dataset, inputs['tf_x_labels'] : tf_batch_labels
        }
        _, l, pred = session.run([optimizer, loss, tf_train_prediction], feed_dict=feed_dict)
        
        # print accuracies on training data, validation data, and test data
        if step % 500 == 0:
            print('Step: %d' % step)
            print('Time elapsed:', time.clock() - start, 's')
            print('Loss: %.2f' % l)
            print('Batch training accuracy: %.1f%%' % accuracy(pred, tf_batch_labels))
            print('Batch validation accuracy: %.1f%%' % accuracy(tf_valid_prediction.eval(), valid_labels))
            print('------')
    print('Total test accuracy: %.1f%%' % accuracy(tf_test_prediction.eval(), test_labels))
    print('Total training time:', time.clock() - start, 's')

graph initialized!
training started!

Step: 0
Time elapsed: 0.121275 s
Loss: 1056.77
Batch training accuracy: 7.8%
Batch validation accuracy: 24.2%
------
Step: 500
Time elapsed: 54.415733 s
Loss: 199.15
Batch training accuracy: 100.0%
Batch validation accuracy: 77.6%
------
Step: 1000
Time elapsed: 107.597122 s
Loss: 121.44
Batch training accuracy: 100.0%
Batch validation accuracy: 77.9%
------
Step: 1500
Time elapsed: 161.629169 s
Loss: 74.08
Batch training accuracy: 100.0%
Batch validation accuracy: 78.1%
------
Step: 2000
Time elapsed: 217.683953 s
Loss: 45.34
Batch training accuracy: 100.0%
Batch validation accuracy: 77.9%
------
Step: 2500
Time elapsed: 277.605018 s
Loss: 27.82
Batch training accuracy: 100.0%
Batch validation accuracy: 78.1%
------
Step: 3000
Time elapsed: 342.250145 s
Loss: 17.12
Batch training accuracy: 100.0%
Batch validation accuracy: 78.2%
------
Step: 3500
Time elapsed: 407.12332 s
Loss: 10.76
Batch training accuracy: 100.0%
Batch validation accuracy: 78.1%

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [30]:
# 3 layer neural network

def build_layer(layer, weights, biases, hidden=True):
    new_layer = tf.matmul(layer, weights) + biases
    if hidden:
        new_layer = tf.nn.relu(new_layer)
    return new_layer
    

def build_logits(inputs_dict, weights_dict, biases_dict, dropout=False, valid=False, test=False):
    if dropout:
        layer_1 = build_layer(inputs_dict['x'] , weights_dict['w1'], biases_dict['b1'])
        layer_1 = tf.nn.dropout(layer_1, 0.5)
        layer_2 = build_layer(layer_1, weights_dict['w2'], biases_dict['b2'])
        layer_2 = tf.nn.dropout(layer_2, 0.5)
        layer_3 = build_layer(layer_2, weights_dict['w3'], biases_dict['b3'])
        layer_3 = tf.nn.dropout(layer_3, 0.5)
        output_layer = build_layer(layer_3, weights_dict['w4'], biases_dict['b4'], hidden=False)
        return output_layer
    elif valid:
        layer_1 = build_layer(inputs_dict['valid_set'] , weights_dict['w1'], biases_dict['b1'])
        layer_2 = build_layer(layer_1, weights_dict['w2'], biases_dict['b2'])
        layer_3 = build_layer(layer_2, weights_dict['w3'], biases_dict['b3'])
        output_layer = build_layer(layer_3, weights_dict['w4'], biases_dict['b4'], hidden=False)
        return output_layer
    elif test:
        layer_1 = build_layer(inputs_dict['test_set'] , weights_dict['w1'], biases_dict['b1'])
        layer_2 = build_layer(layer_1, weights_dict['w2'], biases_dict['b2'])
        layer_3 = build_layer(layer_2, weights_dict['w3'], biases_dict['b3'])
        output_layer = build_layer(layer_3, weights_dict['w4'], biases_dict['b4'], hidden=False)
        return output_layer
    else: 
        layer_1 = build_layer(inputs_dict['x'] , weights_dict['w1'], biases_dict['b1'])
        layer_2 = build_layer(layer_1, weights_dict['w2'], biases_dict['b2'])
        layer_3 = build_layer(layer_2, weights_dict['w3'], biases_dict['b3'])
        output_layer = build_layer(layer_3, weights_dict['w4'], biases_dict['b4'], hidden=False)
        return output_layer
    
nodes_1 = 1024
nodes_2 = 300
nodes_3 = 50
num_labels = 10
beta = 0.0005

graph = tf.Graph()
with graph.as_default():
    # inputs
    inputs_dict = {
        'x' : tf.placeholder(tf.float32, shape=(batch_size, img_size * img_size)),
        'x_labels' : tf.placeholder(tf.float32, shape=(batch_size, num_labels)),
        'valid_set' : valid_dataset,
        'test_set' : test_dataset 
    }
    
    # variables
    # weights randomly initialized with normal distribution
    weights_dict = {
        'w1' : tf.Variable(tf.truncated_normal([img_size * img_size, nodes_1], 
                                               stddev=np.sqrt(2.0/(img_size * img_size)))),
        'w2' : tf.Variable(tf.truncated_normal([nodes_1, nodes_2], 
                                               stddev=np.sqrt(2.0/(nodes_1)))),
        'w3' : tf.Variable(tf.truncated_normal([nodes_2, nodes_3], 
                                               stddev=np.sqrt(2.0/(nodes_2)))),
        'w4' : tf.Variable(tf.truncated_normal([nodes_3, num_labels], 
                                               stddev=np.sqrt(2.0/(nodes_3))))
    }
    # initialize biases to zero
    biases_dict = {
        'b1' : tf.Variable(tf.zeros([nodes_1])),
        'b2' : tf.Variable(tf.zeros([nodes_2])),
        'b3' : tf.Variable(tf.zeros([nodes_3])),
        'b4' : tf.Variable(tf.zeros([num_labels]))
    }
    
    # computations
    train_logits = build_logits(inputs_dict, weights_dict, biases_dict, dropout=True)
    eval_logits = build_logits(inputs_dict, weights_dict, biases_dict)
    
    #loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        labels=inputs_dict['x_labels'], logits=train_logits)) + beta * (
        tf.nn.l2_loss(weights_dict['w1']) + 
        tf.nn.l2_loss(weights_dict['w2']) + 
        tf.nn.l2_loss(weights_dict['w3']) + 
        tf.nn.l2_loss(weights_dict['w4']))
    
    # optimizer
    initial_learning_rate = 0.03
    global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
    # decay every 1000 steps with a base of 0.96, staircase --> discrete decaying steps 
    learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, 5000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # predictions
    tf_train_prediction = tf.nn.softmax(eval_logits)
    tf_valid_prediction = tf.nn.softmax(build_logits(inputs_dict, weights_dict, biases_dict, valid=True))
    tf_test_prediction = tf.nn.softmax(build_logits(inputs_dict, weights_dict, biases_dict, test=True))
print('graph initialized!')

num_steps = 20001
batch_size = 256 

# run the network
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('training started!\n')
    start = time.clock()
    for step in range(num_steps):
        offset = (batch_size * step) % (train_dataset.shape[0] - batch_size)
        tf_batch_dataset = train_dataset[offset:offset + batch_size, :]
        tf_batch_labels = train_labels[offset:offset + batch_size, :]
        feed_dict = {
            inputs_dict['x'] : tf_batch_dataset, 
            inputs_dict['x_labels'] : tf_batch_labels
        }
        _, l, pred = session.run([optimizer, loss, tf_train_prediction], feed_dict=feed_dict)
        
        # print accuracies on training data, validation data, and test data
        if step % 500 == 0:
            print('Step: %d' % step)
            print('Time elapsed:', time.clock() - start, 's')
            print('Loss: %.2f' % l)
            # print('Learning rate: %.1f' % lr)
            print('Batch training accuracy: %.1f%%' % accuracy(pred, tf_batch_labels))
            print('Batch validation accuracy: %.1f%%' % accuracy(tf_valid_prediction.eval(), valid_labels))
            print('------')
    print('Total test accuracy: %.1f%%' % accuracy(tf_test_prediction.eval(), test_labels))
    print('Total training time:', time.clock() - start, 's')
    

graph initialized!
training started!

Step: 0
Time elapsed: 0.294073 s
Loss: 3.40
Batch training accuracy: 7.4%
Batch validation accuracy: 10.9%
------
Step: 500
Time elapsed: 148.296131 s
Loss: 1.50
Batch training accuracy: 82.8%
Batch validation accuracy: 80.5%
------
Step: 1000
Time elapsed: 304.179636 s
Loss: 1.29
Batch training accuracy: 82.4%
Batch validation accuracy: 82.5%
------
Step: 1500
Time elapsed: 455.042243 s
Loss: 1.23
Batch training accuracy: 83.6%
Batch validation accuracy: 83.1%
------
Step: 2000
Time elapsed: 610.560785 s
Loss: 1.27
Batch training accuracy: 80.5%
Batch validation accuracy: 83.5%
------
Step: 2500
Time elapsed: 766.371947 s
Loss: 1.22
Batch training accuracy: 81.6%
Batch validation accuracy: 84.0%
------
Step: 3000
Time elapsed: 919.21553 s
Loss: 1.07
Batch training accuracy: 88.7%
Batch validation accuracy: 84.2%
------
Step: 3500
Time elapsed: 1075.155747 s
Loss: 1.10
Batch training accuracy: 87.9%
Batch validation accuracy: 84.5%
------
Step: 400